In [1]:
import os
import asyncio
from typing import List, Literal, Optional
from pydantic import BaseModel, Field

from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.messages import TextMessage, StructuredMessage
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.tools import FunctionTool

from tavily import TavilyClient, MissingAPIKeyError, InvalidAPIKeyError, UsageLimitExceededError, BadRequestError

# Create model client
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=os.getenv("OPENAI_API_KEY"),  # Uncomment and set your API key if needed
)

# Initialize Tavily client with proper error handling
def get_tavily_client():
    tavily_api_key = os.getenv("TAVILY_API_KEY")
    # tavily_api_key = ""
    if not tavily_api_key:
        print("Warning: TAVILY_API_KEY environment variable not set. Using mock data.")
        return None
    return TavilyClient(api_key=tavily_api_key)

tavily_client = get_tavily_client()

In [11]:
# Define structured output models for our agents
class PainPointAnalysisResult(BaseModel):
    """Result of pain point analysis"""
    pain_points: List[str] = Field(description="List of identified pain points")
    categorization: str = Field(description="Categorization of pain points as a text description")
    prioritization: List[str] = Field(description="Prioritized list of pain points")
    examples: List[str] = Field(description="Real-world examples of these pain points")
    reasoning: str = Field(description="Reasoning behind the analysis")
    
class CompetitorAnalysisResult(BaseModel):
    """Result of competitor analysis"""
    competitors: List[str] = Field(description="List of key competitors")
    strengths: str = Field(description="Strengths of each competitor as a text description")
    weaknesses: str = Field(description="Weaknesses of each competitor as a text description")
    positioning: str = Field(description="Market positioning of each competitor as a text description")
    differentiation_opportunities: List[str] = Field(description="Opportunities for differentiation")
    reasoning: str = Field(description="Reasoning behind the analysis")

class PersonaResearchResult(BaseModel):
    """Result of persona research"""
    persona_name: str = Field(description="Name of the target persona")
    demographics: str = Field(description="Demographics of the target persona")
    behaviors: str = Field(description="Behaviors and habits of the target persona")
    needs: str = Field(description="Needs and goals of the target persona")
    online_communities: List[str] = Field(description="Online communities where this persona can be found")
    reasoning: str = Field(description="Reasoning behind identifying this persona")

class MarketResearchReport(BaseModel):
    """Final market research report"""
    target_persona: PersonaResearchResult = Field(description="Target persona information")
    pain_points: PainPointAnalysisResult = Field(description="Pain point analysis")
    competitors: CompetitorAnalysisResult = Field(description="Competitor analysis")
    recommendations: List[str] = Field(description="Recommendations based on the research")
    summary: str = Field(description="Executive summary of the research")


In [12]:
# Enhance the web_search function to provide better mock data
async def web_search(query: str, search_depth: Literal["basic", "advanced"]) -> str:
    """
    Search the web for information using Tavily API
    
    Args:
        query: The search query
        search_depth: Either "basic" (faster, 1 credit) or "advanced" (more thorough, 2 credits)
    
    Returns:
        Formatted search results as a string
    """
    if not tavily_client:
        # Enhanced mock data based on the query
        if "persona" in query.lower() or "target audience" in query.lower():
            return f"""Mock search results for: {query}

Summary: Research indicates that IT Directors and CTOs in mid-sized companies (100-1000 employees) are key decision-makers for business technology optimization services. They typically have 10+ years of experience, are aged 35-55, and face challenges with legacy system integration and digital transformation.

Sources:
1. The Modern IT Director: Challenges and Opportunities
   URL: https://example.com/it-director-research
   Snippet: IT Directors in mid-sized companies report spending 60% of their time on maintenance rather than innovation. Key pain points include budget constraints, talent shortages, and keeping up with rapid technological changes...

2. CTO Survey 2023: Technology Decision Making
   URL: https://example.com/cto-survey-2023
   Snippet: 78% of CTOs cite "optimizing existing technology investments" as a top priority. They typically evaluate 3-5 vendors before making significant technology decisions...

3. B2B Technology Buying Behavior
   URL: https://example.com/b2b-tech-buying
   Snippet: Technology purchase decisions in mid-sized companies typically involve 6-10 stakeholders, with the IT Director or CTO having the most influence. The buying cycle averages 3-6 months...
"""
        elif "pain point" in query.lower():
            return f"""Mock search results for: {query}

Summary: IT Directors and CTOs in professional services face several key pain points: 1) Integration challenges between legacy and new systems, 2) Difficulty measuring ROI on technology investments, 3) Talent shortages for specialized tech roles, 4) Security concerns with distributed workforces, and 5) Budget constraints limiting innovation.

Sources:
1. Technology Pain Points in Professional Services
   URL: https://example.com/tech-pain-points
   Snippet: A survey of 500 IT leaders revealed that 72% struggle with integrating legacy systems with modern cloud solutions. This creates data silos that hamper decision-making and operational efficiency...

2. The ROI Challenge: Measuring Technology Impact
   URL: https://example.com/tech-roi-measurement
   Snippet: Only 34% of companies report having clear metrics to measure the business impact of their technology investments. This creates challenges in justifying further spending...

3. IT Talent Gap Analysis 2023
   URL: https://example.com/it-talent-gap
   Snippet: 68% of IT Directors report difficulty hiring and retaining specialized technical talent, particularly in cybersecurity, data science, and cloud architecture...
"""
        elif "competitor" in query.lower():
            return f"""Mock search results for: {query}

Summary: The business technology optimization consulting space has several key players including Accenture, Deloitte Digital, IBM Consulting, Capgemini, and boutique firms like Slalom and West Monroe. Large firms offer comprehensive services but are often expensive and less agile, while boutique firms provide more personalized service but may lack global reach.

Sources:
1. Technology Consulting Market Analysis 2023
   URL: https://example.com/tech-consulting-market
   Snippet: The global technology optimization consulting market is valued at $195 billion and growing at 8.3% annually. Major players control 65% of market share, with mid-sized and boutique firms gaining ground by specializing in specific industries...

2. Consulting Firm Comparison: Enterprise vs. Boutique
   URL: https://example.com/consulting-comparison
   Snippet: While Accenture and Deloitte dominate with comprehensive service offerings, boutique firms like Slalom report higher client satisfaction scores (NPS of 72 vs. 64 for larger firms)...

3. Technology Consulting Pricing Models
   URL: https://example.com/consulting-pricing
   Snippet: Enterprise consulting firms typically charge 30-50% more than mid-sized competitors. However, they offer broader capabilities and greater resources for complex, global implementations...
"""
        else:
            return f"""Mock search results for: {query}

Summary: The search results provide information relevant to {query}, highlighting key trends, challenges, and opportunities in this area.

Sources:
1. Recent Research on {query.title()}
   URL: https://example.com/research-{query.replace(' ', '-')}
   Snippet: Studies show that {query} is becoming increasingly important in the business landscape, with organizations investing more resources in this area...

2. Best Practices for {query.title()}
   URL: https://example.com/best-practices-{query.replace(' ', '-')}
   Snippet: Industry leaders recommend a strategic approach to {query}, focusing on measurable outcomes and stakeholder alignment...

3. Future Trends in {query.title()}
   URL: https://example.com/trends-{query.replace(' ', '-')}
   Snippet: Experts predict significant evolution in {query} over the next 3-5 years, with emerging technologies playing a crucial role...
"""
    
    try:
        # Original Tavily implementation remains unchanged
        # Convert to synchronous call since Tavily doesn't have async API
        response = tavily_client.search(
            query=query,
            search_depth=search_depth,
            include_answer=True,
            max_results=5
        )
        
        results = f"Search results for: {query}\n\n"
        
        # Add the AI-generated answer if available
        if response.get("answer"):
            results += f"Summary: {response.get('answer')}\n\n"
        
        results += "Sources:\n"
        for i, result in enumerate(response.get("results", []), 1):
            title = result.get("title", "No title")
            url = result.get("url", "No URL")
            content_snippet = result.get("content", "")[:200] + "..." if result.get("content") else ""
            
            results += f"{i}. {title}\n"
            results += f"   URL: {url}\n"
            results += f"   Snippet: {content_snippet}\n\n"
        
        return results
    except (MissingAPIKeyError, InvalidAPIKeyError) as e:
        return f"API key error: {str(e)}\nPlease check your Tavily API key configuration."
    except UsageLimitExceededError:
        return f"Tavily API usage limit exceeded. Consider upgrading your plan."
    except Exception as e:
        return f"Error performing search: {str(e)}"


async def find_communities(persona: str, platform: str) -> str:
    """
    Find online communities for a specific persona
    
    Args:
        persona: The target persona/audience
        platform: Optional specific platform to search for communities
    
    Returns:
        List of relevant communities
    """
    if not tavily_client:
        # Fallback to mock data
        communities = f"Communities for {persona}:\n"
        communities += f"- Reddit r/{persona.lower().replace(' ', '')}\n"
        communities += "- LinkedIn groups for professionals\n"
        communities += "- Facebook industry groups\n"
        return communities
    
    try:
        platform_str = f"on {platform}" if platform else ""
        query = f"online communities for {persona} {platform_str} discussion forums groups"
        
        response = tavily_client.search(
            query=query,
            search_depth="basic",
            include_answer=True,
            max_results=5
        )
        
        results = f"Communities for {persona}{' on ' + platform if platform else ''}:\n\n"
        
        # Add the AI-generated answer if available
        if response.get("answer"):
            results += f"{response.get('answer')}\n\n"
        
        results += "Sources:\n"
        for i, result in enumerate(response.get("results", []), 1):
            title = result.get("title", "No title")
            url = result.get("url", "No URL")
            
            results += f"{i}. {title}\n"
            results += f"   URL: {url}\n\n"
        
        return results
    except Exception as e:
        # Fallback to mock data if search fails
        fallback = f"Communities for {persona}:\n"
        fallback += f"- Reddit r/{persona.lower().replace(' ', '')}\n"
        fallback += "- LinkedIn groups for professionals\n"
        fallback += "- Facebook industry groups\n"
        fallback += f"Note: Using fallback data due to search error: {str(e)}"
        return fallback

async def industry_trends(industry: str, timeframe: str) -> str:
    """
    Search for industry trends in a specific timeframe
    
    Args:
        industry: The industry to research
        timeframe: Time period for trends (recent, yearly, etc.)
    
    Returns:
        Formatted trend information
    """
    if not tavily_client:
        return f"Mock {timeframe} trends in {industry} industry:\n- Trend 1: Example trend\n- Trend 2: Another trend"
    
    try:
        query = f"latest trends in {industry} industry {timeframe}"
        response = tavily_client.search(
            query=query,
            search_depth="basic",
            include_answer=True,
            max_results=5
        )
        
        results = f"{timeframe.capitalize()} trends in {industry}:\n\n"
        
        # Add the AI-generated answer if available
        if response.get("answer"):
            results += f"{response.get('answer')}\n\n"
        
        results += "Sources:\n"
        for i, result in enumerate(response.get("results", []), 1):
            title = result.get("title", "No title")
            url = result.get("url", "No URL")
            
            results += f"{i}. {title}\n"
            results += f"   URL: {url}\n\n"
        
        return results
    except Exception as e:
        return f"Error searching for industry trends: {str(e)}"

# Create function tools with the updated implementations
web_search_tool = FunctionTool(web_search, description="Search the web for information", strict=True)
find_communities_tool = FunctionTool(find_communities, description="Find online communities for a specific persona", strict=True)
industry_trends_tool = FunctionTool(industry_trends, description="Research trends in a specific industry", strict=True)

In [13]:
# Add a function to create mock data for testing
def create_mock_research_report():
    """Create a mock market research report for testing purposes"""
    persona = PersonaResearchResult(
        persona_name="IT Director/CTO",
        demographics="35-55 years old, primarily male (68%), bachelor's or master's degree in computer science or business, 10+ years in IT management",
        behaviors="Regularly attends industry conferences, consumes content via LinkedIn and specialized IT publications, makes decisions based on ROI and security considerations",
        needs="Needs to optimize IT spending, improve system integration, enhance security, and demonstrate business value of technology investments",
        online_communities=["Spiceworks Community", "Reddit r/sysadmin", "LinkedIn IT Leadership Groups", "CIO Forum", "TechRepublic"],
        reasoning="IT Directors and CTOs are primary decision-makers for business technology optimization services in mid-sized companies. They control budgets and seek solutions to improve efficiency while reducing costs."
    )
    
    pain_points = PainPointAnalysisResult(
        pain_points=[
            "Legacy system integration challenges",
            "Difficulty measuring ROI on technology investments",
            "IT talent shortages and retention issues",
            "Security concerns with distributed workforce",
            "Budget constraints limiting innovation",
            "Shadow IT proliferation",
            "Keeping pace with rapid technological change"
        ],
        categorization="Technical: Legacy system integration challenges, Security concerns with distributed workforce. Financial: Budget constraints limiting innovation, Difficulty measuring ROI on technology investments. Organizational: IT talent shortages and retention issues, Shadow IT proliferation. Strategic: Keeping pace with rapid technological change.",
        prioritization=[
            "Legacy system integration challenges",
            "Difficulty measuring ROI on technology investments",
            "IT talent shortages and retention issues",
            "Security concerns with distributed workforce",
            "Budget constraints limiting innovation"
        ],
        examples=[
            "An IT Director at a financial services firm spent 8 months and $2M on a digital transformation project that failed to integrate with legacy systems",
            "A manufacturing company CTO couldn't justify additional technology investments because they lacked metrics showing business impact of previous implementations",
            "A healthcare organization lost three key IT staff in six months due to burnout from maintaining complex legacy systems"
        ],
        reasoning="These pain points were identified based on frequency of mention in industry forums, survey data, and direct quotes from IT leaders. Legacy integration and ROI measurement emerged as the most critical issues based on both frequency and severity."
    )
    
    competitors = CompetitorAnalysisResult(
        competitors=["Accenture", "Deloitte Digital", "IBM Consulting", "Capgemini", "Slalom", "West Monroe"],
        strengths="Accenture: Global reach, comprehensive service offerings, strong industry expertise. Deloitte Digital: Integration of technology and business strategy, strong brand recognition. IBM Consulting: Deep technical expertise, proprietary technology solutions. Capgemini: Strong European presence, digital transformation focus. Slalom: Local market focus, high client satisfaction, agile approach. West Monroe: Industry specialization, mid-market focus, business-technology integration.",
        weaknesses="Accenture: High cost, can be inflexible, sometimes over-engineers solutions. Deloitte Digital: Expensive, sometimes bureaucratic, staff turnover. IBM Consulting: Legacy reputation, sometimes pushes proprietary solutions. Capgemini: Less brand recognition in US, inconsistent service quality. Slalom: Limited global reach, less suitable for enterprise-scale projects. West Monroe: Limited geographic coverage, smaller resource pool.",
        positioning="Accenture: Premium full-service global technology consulting. Deloitte Digital: Business-led digital transformation. IBM Consulting: Enterprise technology modernization. Capgemini: Digital transformation and innovation. Slalom: Local, personalized business and technology consulting. West Monroe: Business-focused technology consulting for the mid-market.",
        differentiation_opportunities=[
            "Offer transparent, outcome-based pricing models rather than traditional time and materials",
            "Develop specialized expertise in legacy-to-cloud integration for specific industries",
            "Create proprietary measurement frameworks that clearly demonstrate ROI",
            "Provide flexible engagement models that scale with client needs",
            "Combine strategic consulting with hands-on implementation capabilities"
        ],
        reasoning="Analysis of competitor websites, client testimonials, and industry reports reveals a market dominated by large firms with comprehensive offerings but often at premium prices and with less flexibility. Mid-sized firms compete on personalization and industry specialization but may lack breadth."
    )
    
    report = MarketResearchReport(
        target_persona=persona,
        pain_points=pain_points,
        competitors=competitors,
        recommendations=[
            "Develop a proprietary framework for measuring technology ROI that addresses the second highest pain point",
            "Create specialized service offerings focused on legacy system integration for financial services and healthcare industries",
            "Implement transparent, outcome-based pricing to differentiate from competitors' traditional models",
            "Establish thought leadership content specifically addressing IT talent optimization",
            "Develop a rapid assessment tool that quickly identifies integration challenges and potential solutions"
        ],
        summary="This market research report identifies IT Directors and CTOs in mid-sized companies as the primary target persona for business technology optimization consulting. The research uncovered 7 key pain points, with legacy system integration and ROI measurement being most critical. Analysis of 6 major competitors revealed opportunities to differentiate through specialized integration expertise, transparent pricing models, and proprietary ROI measurement frameworks."
    )
    
    return report

# Add a function to test with mock data
def test_with_mock_data():
    """Run a test using mock data instead of the full workflow"""
    report = create_mock_research_report()
    
    # Print the final report in a structured format
    print("\n=== MOCK MARKET RESEARCH REPORT ===")
    print(f"EXECUTIVE SUMMARY:\n{report.summary}\n")
    
    print("TARGET PERSONA:")
    print(f"Name: {report.target_persona.persona_name}")
    print(f"Demographics: {report.target_persona.demographics}")
    print(f"Behaviors: {report.target_persona.behaviors}")
    print(f"Needs: {report.target_persona.needs}")
    print(f"Online Communities: {', '.join(report.target_persona.online_communities)}")
    
    print("\nKEY PAIN POINTS:")
    for i, pain in enumerate(report.pain_points.prioritization[:3], 1):
        print(f"{i}. {pain}")
    
    print("\nCOMPETITOR ANALYSIS:")
    for comp in report.competitors.competitors[:3]:
        print(f"- {comp}")
        print(f"  Strengths: {report.competitors.strengths.get(comp, 'N/A')}")
        print(f"  Weaknesses: {report.competitors.weaknesses.get(comp, 'N/A')}")
    
    print("\nRECOMMENDATIONS:")
    for i, rec in enumerate(report.recommendations, 1):
        print(f"{i}. {rec}")
    
    return report

In [14]:
# Create the agents
market_research_manager = UserProxyAgent(
    name="Market_Research_Manager",
    description="You are a market research manager responsible for coordinating research tasks and presenting findings. You'll work with specialized agents to conduct comprehensive market research.",
)

persona_researcher = AssistantAgent(
    name="Persona_Researcher",
    model_client=model_client,
    tools=[web_search_tool, find_communities_tool],  # Use the strict function tools
    system_message="You are an expert in identifying target personas and gathering information about their demographics, behaviors, and needs. Use web search to find relevant information and identify online communities where these personas hang out.",
    output_content_type=PersonaResearchResult,
)

pain_point_analyzer = AssistantAgent(
    name="PainPoint_Analyzer",
    model_client=model_client,
    tools=[web_search_tool],  # Use the strict function tool
    system_message="You are skilled at analyzing text data to identify common pain points, frustrations, and unmet needs. Analyze discussions in online communities to identify common pain points, categorize them, and prioritize based on frequency and severity.",
    output_content_type=PainPointAnalysisResult,
)

competitor_analyzer = AssistantAgent(
    name="Competitor_Analyzer",
    model_client=model_client,
    tools=[web_search_tool],  # Use the strict function tool
    system_message="You are an expert in analyzing competitor websites and marketing materials to identify their strengths, weaknesses, and positioning. Identify key competitors in the market and analyze their offerings to find opportunities for differentiation.",
    output_content_type=CompetitorAnalysisResult,
)

In [15]:
# Main function to run the market research workflow
async def conduct_market_research(industry: str, product_service: str):
    # Initialize the cancellation token
    cancellation_token = CancellationToken()
    
    print("=== Starting Market Research Process ===")
    
    try:
        # Step 1: Persona Research
        print("\n=== Step 1: Persona Research ===")
        persona_task = f"Identify potential target personas for a {product_service} in the {industry} industry. Research their demographics, behaviors, and needs."
        persona_response = await persona_researcher.on_messages(
            [TextMessage(content=persona_task, source=market_research_manager.name)],
            cancellation_token=cancellation_token,
        )
        persona_result = persona_response.chat_message.content
        print(f"Persona Research Complete: {persona_result.persona_name}")
        
        # Step 2: Pain Point Analysis
        print("\n=== Step 2: Pain Point Analysis ===")
        pain_point_task = f"Analyze the pain points for the {persona_result.persona_name} persona in the {industry} industry, particularly related to {product_service}. Use the communities identified: {', '.join(persona_result.online_communities)}"
        
        try:
            pain_point_response = await pain_point_analyzer.on_messages(
                [TextMessage(content=pain_point_task, source=market_research_manager.name)],
                cancellation_token=cancellation_token,
            )
            pain_point_result = pain_point_response.chat_message.content
            print(f"Pain Point Analysis Complete: {len(pain_point_result.pain_points)} pain points identified")
        except Exception as e:
            print(f"Error in Pain Point Analysis: {str(e)}")
            print("Using mock pain point data instead")
            pain_point_result = create_mock_research_report().pain_points
        
        # Step 3: Competitor Analysis
        print("\n=== Step 3: Competitor Analysis ===")
        competitor_task = f"Identify and analyze key competitors offering {product_service} to {persona_result.persona_name} in the {industry} industry. Focus on their strengths, weaknesses, and positioning."
        
        try:
            competitor_response = await competitor_analyzer.on_messages(
                [TextMessage(content=competitor_task, source=market_research_manager.name)],
                cancellation_token=cancellation_token,
            )
            competitor_result = competitor_response.chat_message.content
            print(f"Competitor Analysis Complete: {len(competitor_result.competitors)} competitors analyzed")
        except Exception as e:
            print(f"Error in Competitor Analysis: {str(e)}")
            print("Using mock competitor data instead")
            competitor_result = create_mock_research_report().competitors
        
        # Step 4: Generate Final Report
        print("\n=== Step 4: Generating Final Report ===")
        
        # Extract the first pain point safely
        top_pain_point = pain_point_result.prioritization[0] if pain_point_result.prioritization else "identified pain points"
        
        # Extract the first differentiation opportunity safely
        top_diff_opp = competitor_result.differentiation_opportunities[0] if competitor_result.differentiation_opportunities else "identified differentiation opportunities"
        
        final_report = MarketResearchReport(
            target_persona=persona_result,
            pain_points=pain_point_result,
            competitors=competitor_result,
            recommendations=[
                f"Focus on solving the top pain point: {top_pain_point}",
                f"Differentiate from competitors by: {top_diff_opp}",
                "Develop targeted messaging that resonates with the persona's needs",
                "Create content that addresses specific pain points identified"
            ],
            summary=f"This market research report identifies {persona_result.persona_name} as the primary target persona for {product_service} in the {industry} industry. The research uncovered {len(pain_point_result.pain_points)} key pain points and analyzed {len(competitor_result.competitors)} major competitors, revealing several opportunities for differentiation and market positioning."
        )
        
        print("\n=== Market Research Complete ===")
        return final_report
    
    except Exception as e:
        print(f"\n=== Error in Market Research Process: {str(e)} ===")
        print("Falling back to mock data...")
        return create_mock_research_report()

In [16]:
# Define these variables at the module level so they can be accessed by both functions
INDUSTRY = "Small town Professional Services like a lawyer, financial planner, people that have to use a computer for work, but aren't really tech savvy, sales people etc."
PRODUCT_SERVICE = "Remote access IT helpdesk support services"

# Example usage
async def main():
    report = await conduct_market_research(INDUSTRY, PRODUCT_SERVICE)
    
    # Print the final report in a structured format
    print("\n=== FINAL MARKET RESEARCH REPORT ===")
    print(f"EXECUTIVE SUMMARY:\n{report.summary}\n")
    
    print("TARGET PERSONA:")
    print(f"Name: {report.target_persona.persona_name}")
    print(f"Demographics: {report.target_persona.demographics}")
    print(f"Behaviors: {report.target_persona.behaviors}")
    print(f"Needs: {report.target_persona.needs}")
    print(f"Online Communities: {', '.join(report.target_persona.online_communities)}")
    
    print("\nKEY PAIN POINTS:")
    for i, pain in enumerate(report.pain_points.prioritization[:3], 1):
        print(f"{i}. {pain}")
    
    print("\nCOMPETITOR ANALYSIS:")
    for comp in report.competitors.competitors[:3]:
        print(f"- {comp}")
        # Since strengths and weaknesses are now strings, not dictionaries
        print(f"  Strengths: {report.competitors.strengths}")
        print(f"  Weaknesses: {report.competitors.weaknesses}")
    
    print("\nRECOMMENDATIONS:")
    for i, rec in enumerate(report.recommendations, 1):
        print(f"{i}. {rec}")
    
    # Close the model client
    await model_client.close()

# For Jupyter notebooks, provide a synchronous version
def run_research_sync(use_mock=False):
    # Import required here to avoid circular imports
    import asyncio
    import nest_asyncio
    import time
    
    # Apply nest_asyncio to allow nested event loops in Jupyter
    nest_asyncio.apply()
    
    # If use_mock is True, just return mock data without running the full workflow
    if use_mock:
        print("Using mock data as requested...")
        return test_with_mock_data()
    
    # Get the current event loop
    loop = asyncio.get_event_loop()
    
    try:
        # Run the conduct_market_research function with the global variables
        return loop.run_until_complete(conduct_market_research(INDUSTRY, PRODUCT_SERVICE))
    except RuntimeError as e:
        print(f"Runtime error: {e}")
        print("Trying alternative approach for Jupyter environment...")
        # Alternative approach if the above fails
        from IPython.lib.backgroundjobs import BackgroundJobManager
        jobs = BackgroundJobManager()
        result = [None]
        
        def run_async():
            result[0] = asyncio.run(conduct_market_research(INDUSTRY, PRODUCT_SERVICE))
            
        jobs.new(run_async)
        while result[0] is None:
            time.sleep(0.1)
        return result[0]
    except Exception as e:
        print(f"Error: {str(e)}")
        print("Falling back to mock data...")
        return test_with_mock_data()

In [17]:
def format_market_research_report(report):
    """
    Format a MarketResearchReport object into a well-structured, readable document
    
    Args:
        report: A MarketResearchReport object containing the research data
        
    Returns:
        A formatted string containing the report in markdown format
    """
    # Create the markdown document
    markdown = "# Market Research Report\n\n"
    
    # Executive Summary section
    markdown += "## Executive Summary\n\n"
    markdown += f"{report.summary}\n\n"
    
    # Target Persona section
    markdown += "## Target Persona: " + report.target_persona.persona_name + "\n\n"
    markdown += "### Demographics\n"
    markdown += f"{report.target_persona.demographics}\n\n"
    
    markdown += "### Behaviors\n"
    markdown += f"{report.target_persona.behaviors}\n\n"
    
    markdown += "### Needs\n"
    markdown += f"{report.target_persona.needs}\n\n"
    
    markdown += "### Online Communities\n"
    markdown += "\n".join([f"- {community}" for community in report.target_persona.online_communities]) + "\n\n"
    
    markdown += "### Reasoning\n"
    markdown += f"{report.target_persona.reasoning}\n\n"
    
    # Pain Points section
    markdown += "## Pain Point Analysis\n\n"
    
    markdown += "### Key Pain Points\n"
    markdown += "\n".join([f"- {pain}" for pain in report.pain_points.pain_points]) + "\n\n"
    
    markdown += "### Categorization\n"
    markdown += f"{report.pain_points.categorization}\n\n"
    
    markdown += "### Prioritized Pain Points\n"
    markdown += "\n".join([f"{i+1}. {pain}" for i, pain in enumerate(report.pain_points.prioritization)]) + "\n\n"
    
    markdown += "### Real-world Examples\n"
    markdown += "\n".join([f"- {example}" for example in report.pain_points.examples]) + "\n\n"
    
    markdown += "### Reasoning\n"
    markdown += f"{report.pain_points.reasoning}\n\n"
    
    # Competitor Analysis section
    markdown += "## Competitor Analysis\n\n"
    
    markdown += "### Key Competitors\n"
    markdown += "\n".join([f"- {competitor}" for competitor in report.competitors.competitors]) + "\n\n"
    
    markdown += "### Strengths\n"
    markdown += f"{report.competitors.strengths}\n\n"
    
    markdown += "### Weaknesses\n"
    markdown += f"{report.competitors.weaknesses}\n\n"
    
    markdown += "### Market Positioning\n"
    markdown += f"{report.competitors.positioning}\n\n"
    
    markdown += "### Differentiation Opportunities\n"
    markdown += "\n".join([f"- {opportunity}" for opportunity in report.competitors.differentiation_opportunities]) + "\n\n"
    
    markdown += "### Reasoning\n"
    markdown += f"{report.competitors.reasoning}\n\n"
    
    # Recommendations section
    markdown += "## Recommendations\n\n"
    markdown += "\n".join([f"{i+1}. {recommendation}" for i, recommendation in enumerate(report.recommendations)]) + "\n\n"
    
    return markdown

# Example usage
def display_formatted_report(report=None):
    """
    Display a formatted market research report
    
    Args:
        report: Optional MarketResearchReport object. If None, creates a mock report.
    """
    if report is None:
        report = create_mock_research_report()
    
    formatted_report = format_market_research_report(report)
    
    # In a Jupyter notebook, you can display markdown directly
    from IPython.display import Markdown, display
    display(Markdown(formatted_report))
    
    # Alternatively, print the markdown (less pretty but works everywhere)
    # print(formatted_report)
    
    return formatted_report

In [18]:
# To display the report from a previous research run:
report = run_research_sync()
display_formatted_report(report)

# Or to display a mock report for testing:
# display_formatted_report()

# Testing
# run_research_sync(use_mock=True)
# test_with_mock_data()

=== Starting Market Research Process ===

=== Step 1: Persona Research ===
Persona Research Complete: Small Town Professional Services IT User

=== Step 2: Pain Point Analysis ===
Pain Point Analysis Complete: 5 pain points identified

=== Step 3: Competitor Analysis ===
Competitor Analysis Complete: 6 competitors analyzed

=== Step 4: Generating Final Report ===

=== Market Research Complete ===


# Market Research Report

## Executive Summary

This market research report identifies Small Town Professional Services IT User as the primary target persona for Remote access IT helpdesk support services in the Small town Professional Services like a lawyer, financial planner, people that have to use a computer for work, but aren't really tech savvy, sales people etc. industry. The research uncovered 5 key pain points and analyzed 6 major competitors, revealing several opportunities for differentiation and market positioning.

## Target Persona: Small Town Professional Services IT User

### Demographics
Typically aged 30-55, predominantly male and female professionals working in fields such as law, finance, and sales. Many are owners or employees of small businesses in towns with populations ranging from 10,000 to 50,000. They have at least a college degree and are moderately tech-savvy.

### Behaviors
These professionals rely heavily on their computers for daily business operations but are not deeply familiar with tech troubleshooting. They prefer using easily navigable tech solutions and value customer support that is responsive and tailored to non-specialists. They often use multiple platforms, sometimes causing challenges in system management.

### Needs
Key needs include reliable and budget-friendly IT support services that ensure business continuity. They require quick resolution of tech issues, robust data security, and easy-to-understand tech updates and maintenance. Compliance with any industry-specific tech guidelines is also important to them.

### Online Communities
- Remote IT Helpdesk Professionals LinkedIn group
- Digital Nomads Facebook group
- r/JobsSA on Reddit

### Reasoning
Small town professionals in fields like law and finance rely on technology but often lack deeper tech expertise, making them ideal candidates for remote IT helpdesk services. Their need for responsive, understandable, and cost-effective tech solutions aligns perfectly with the value propositions offered by remote IT support services.

## Pain Point Analysis

### Key Pain Points
- Slow access to remote IT support tools, especially in areas with poor internet connectivity.
- Limited or nonexistent local helpdesk support leading to dependency on external remote services.
- Communication challenges with remote IT support due to time zone differences and lack of reliable contact platforms.
- Lack of understanding or training in using remote access tools effectively.
- Security concerns about remote access, especially regarding sensitive client data in professional services like law and finance.

### Categorization
The pain points are primarily categorized into issues of technology accessibility, communication barriers, user skill gaps, and security concerns.

### Prioritized Pain Points
1. 1. Slow access to remote IT support tools.
2. 2. Limited local helpdesk support.
3. 3. Communication challenges with support teams.
4. 4. Lack of training in remote access tools.
5. 5. Security concerns regarding data privacy.

### Real-world Examples
- Complaints in LinkedIn groups about delays in accessing support services.
- Digital nomads on Facebook discussing difficulties in coordinating with IT support across time zones.
- Reddit users expressing concerns about the initial training and use of remote IT support tools.

### Reasoning
The primary pain points arise from the rural and potentially technologically underserved setting of small towns, compounded by the professional, yet non-tech-savvy nature of the users, leading to frustration with remote access tools and support services. The prioritization considers both frequency (slow access is a common theme) and potential severity of impact (security concerns in professional services).

## Competitor Analysis

### Key Competitors
- Landon Technologies
- GoMyITGuy
- Power Consulting
- VirtuWorks
- ITns Group
- eSudo

### Strengths
Landon Technologies offers 24/7 helpdesk service, which is crucial for continuous operations and immediate issue resolution. GoMyITGuy provides quick access to experienced professionals, emphasizing affordability and reliability. Power Consulting combines remote and on-site support, offering flexible solutions adaptable to client needs. VirtuWorks focuses on compliance and productivity, ensuring data protection tailored for legal and financial sectors. ITns Group supports both remote and onsite services with a focus on rapid response times. eSudo emphasizes proactive rather than reactive IT support, focusing on regular updates and maintenance.

### Weaknesses
Landon Technologies may not have differentiated services for specific professional sectors like legal or financial industries. GoMyITGuy's strength in affordability may come at the cost of customized support. Power Consulting's on-site support speed might be limited outside major cities. VirtuWorks might not be as competitive for non-legal fields. ITns Group may have a limited geographic presence outside urban areas. eSudo's proactive approach could be more costly, possibly deterring smaller firms.

### Market Positioning
Landon Technologies positions itself as a reliable, 24/7 IT service provider for small businesses with an emphasis on seamless integration. GoMyITGuy focuses on fast, reliable, and cost-efficient service, appealing to cost-conscious clients. Power Consulting markets itself towards companies needing flexible, scalable IT support solutions. VirtuWorks caters specifically to law firms, offering tailored solutions for compliance and productivity. ITns Group is positioned towards rapid service delivery, ideal for small to medium-sized law firms. eSudo targets firms looking for ongoing maintenance and proactive care to prevent IT issues.

### Differentiation Opportunities
- Target niche markets within the professional services sector, such as specialized services for financial planners or real estate agencies.
- Develop unique service offerings tailored to small town businesses, focusing on personal, localized support.
- Introduce scalable pricing models for different firm sizes, offering flexibility and growth adaptation.
- Offer added value with cybersecurity and data protection services specifically designed for compliance, vital for sensitive fields such as law and finance.
- Leverage a mixed model of remote and on-site rapid response service, especially valuable in small towns where major IT firms might have slower response times.

### Reasoning
Analysis shows that while competitors provide strong general IT support, there's room to innovate through sector-specific services, localized and personalized support, and flexible pricing models. Legal and financial industries demand compliance-focused solutions, yet few competitors offer specialized services for small-town businesses. This gap presents an opportunity to differentiate by creating tailored packages that cater to the unique needs of professional services in smaller communities.

## Recommendations

1. Focus on solving the top pain point: 1. Slow access to remote IT support tools.
2. Differentiate from competitors by: Target niche markets within the professional services sector, such as specialized services for financial planners or real estate agencies.
3. Develop targeted messaging that resonates with the persona's needs
4. Create content that addresses specific pain points identified



"# Market Research Report\n\n## Executive Summary\n\nThis market research report identifies Small Town Professional Services IT User as the primary target persona for Remote access IT helpdesk support services in the Small town Professional Services like a lawyer, financial planner, people that have to use a computer for work, but aren't really tech savvy, sales people etc. industry. The research uncovered 5 key pain points and analyzed 6 major competitors, revealing several opportunities for differentiation and market positioning.\n\n## Target Persona: Small Town Professional Services IT User\n\n### Demographics\nTypically aged 30-55, predominantly male and female professionals working in fields such as law, finance, and sales. Many are owners or employees of small businesses in towns with populations ranging from 10,000 to 50,000. They have at least a college degree and are moderately tech-savvy.\n\n### Behaviors\nThese professionals rely heavily on their computers for daily business